<a href="https://colab.research.google.com/github/R2D2-like/fine-tuning_with_custom_dateset_for_whisper/blob/main/customwhisper_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("figfig/whisper-small-en")

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [ ]:
model.config.forced_decoder_ids \
    = processor.get_decoder_prompt_ids(language = "en", task = "transcribe")
model.config.suppress_tokens = []

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install datasets>=2.6.1
from datasets import load_dataset, DatasetDict, Audio
common_voice_test = load_dataset("mozilla-foundation/common_voice_11_0"\
    , "ja", split="test", use_auth_token=True)
common_voice_test = common_voice_test.select(range(1))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16000))

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6505it [00:00, 47381.54it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...

Generating other split: 0 examples [00:00, ? examples/s]





Reading metadata...: 1146it [00:00, 62902.21it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]






Reading metadata...: 2464it [00:00, 49733.47it/s]


Dataset common_voice_11_0 downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/ja/11.0.0/2c65b95d99ca879b1b1074ea197b65e0497848fd697fdb0582e0f6b75b6f4da0. Subsequent calls will reuse this data.


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

# 推論
##HSRに実装するときはspeech dataをcommon_voice_testのデータセットではなく流れてきた音声に変えたい
speech_data = common_voice_test['audio'][0]["array"]
inputs = processor.feature_extractor(speech_data
    , return_tensors="pt", sampling_rate=16_000).input_features.to(device)
predicted_ids = model.generate(inputs, max_length=480_000)
processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]

'そぼを大麦粉をよくサイコロコロして'

In [ ]:
predicted_ids

tensor([[50258, 50266, 50359, 50363,  5040,   487,   120,  5998,  3582,  3994,
            99, 32993,  5998,  5591,  6134, 23607,  8040, 18066, 17164, 18066,
         17164,  8822, 50257]])

In [ ]:
speech_data

array([ 1.2503711e-15,  2.5257222e-15, -7.4783988e-15, ...,
       -3.2072302e-04, -1.5429599e-04,  4.8251968e-04], dtype=float32)

In [ ]:
type(speech_data)

numpy.ndarray